In [5]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class InceptionModule(nn.Module):
    def __init__(self, in_channels, out1x1, red3x3, out3x3, red5x5, out5x5, pool_proj):
        super(InceptionModule, self).__init__()

        # 1x1 conv branch
        self.branch1x1 = nn.Conv2d(in_channels, out1x1, kernel_size=1)

        # 1x1 conv followed by 3x3 conv branch
        self.branch3x3 = nn.Sequential(
            nn.Conv2d(in_channels, red3x3, kernel_size=1),
            nn.Conv2d(red3x3, out3x3, kernel_size=3, padding=1)
        )

        # 1x1 conv followed by 5x5 conv branch
        self.branch5x5 = nn.Sequential(
            nn.Conv2d(in_channels, red5x5, kernel_size=1),
            nn.Conv2d(red5x5, out5x5, kernel_size=5, padding=2)
        )

        # 3x3 pool followed by 1x1 conv branch
        self.branch_pool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, pool_proj, kernel_size=1)
        )

    def forward(self, x):
        # Concatenate the output of each branch
        return torch.cat([self.branch1x1(x), self.branch3x3(x), self.branch5x5(x), self.branch_pool(x)], dim=1)

# Crear un tensor de datos sintéticos para probar la operación Inception
synthetic_data = torch.randn(1, 3, 224, 224)  # Tamaño típico de una imagen

# Crear una instancia de InceptionModule
inception_module = InceptionModule(3, 64, 128, 128, 32, 32, 32)

# Pasar los datos sintéticos a través de la operación Inception
output_synthetic = inception_module(synthetic_data)
print("Salida de la operación Inception con datos sintéticos:")
print(output_synthetic.shape)


Salida de la operación Inception con datos sintéticos:
torch.Size([1, 256, 224, 224])


In [10]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Descargar el conjunto de datos CIFAR-10
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Crear DataLoader para el conjunto de datos de entrenamiento y prueba
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Obtener un lote de datos de entrenamiento para probar la operación Inception
sample_data, _ = next(iter(train_loader))

# Pasar el lote de datos a través de la operación Inception
output_cifar10 = inception_module(sample_data)
print("Salida de la operación Inception con datos de CIFAR-10:")
print(output_cifar10.shape)


Files already downloaded and verified
Files already downloaded and verified
Salida de la operación Inception con datos de CIFAR-10:
torch.Size([32, 256, 32, 32])
